In [0]:

'''
Unificar Evidencias (Tablas staging -> Final)

Este notebook se ejecuta después de todas las tareas de validación
Es idempotente (seguro de ejecutarse varias veces sin errores) y seguro ante fallos

Lógica
1. 'main()' orquesta el proceso.
2. Lee 'table_config' para encontrar todas las tablas 'staging_evidences_table'
3. Las une dinámicamente ('unionByName') para crear un DataFrame 'df_staging_completo'
4. Anti-Duplicación: obtiene los execution_id's de staging y los cruza con
   'dq_evidences' para encontrar los IDs que aún no han sido procesados
5. Filtra 'df_staging_completo' para quedarse solo con los datos nuevos
6. Realiza 'APPEND' de los datos nuevos a la tabla final 'dq_evidences'
7. Limpieza: si el 'APPEND' es exitoso, borra los execution_id's procesados
   de todas las tablas de staging.
'''

# 1. Imports y Widgets
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, broadcast
from functools import reduce
from delta.tables import DeltaTable

# Widgets
def get_mandatory_widget(name, label=None):
    if label is None:
        label = name
    try:
        # Intentar crear el widget
        dbutils.widgets.text(name, "", label)
    except:
        # Si ya existe, ignorar el error
        pass
    value = dbutils.widgets.get(name)
    if not value.strip():
        raise ValueError(f"Parámetro obligatorio: {name}")
    return value

CATALOG = get_mandatory_widget("catalog_name","Catálogo de UC donde residen las tablas")
SCHEMA = get_mandatory_widget("schema_name", "Esquema de UC donde residen las tablas")

TABLE_CONFIG = f"{CATALOG}.{SCHEMA}.dq_tables_config"
EVIDENCES_TABLE = f"{CATALOG}.{SCHEMA}.dq_evidences"

# 2. Función Principal (main)
def main():
    
    try:
        # 1. Leer la configuración maestra        
        master_configs = (spark.table(TABLE_CONFIG)
                          .select("table_name", "staging_evidences_table")
                          .collect())
        
        if not master_configs:
            raise Exception(f"No se encontró configuración de maestros en {TABLE_CONFIG}")

        # 2. Listar todos los DataFrames de staging con evidencias sin procesar
        dfs_to_union = []
        staging_table_paths = []
        
        df_processed_ids = spark.table(EVIDENCES_TABLE).select("execution_id").distinct()

        for config in master_configs:
            staging_table = config.staging_evidences_table
            staging_table_path = f"{CATALOG}.{SCHEMA}.{staging_table}"
            staging_table_paths.append(staging_table_path) 
            
            if spark.catalog.tableExists(staging_table_path):
                df_staging = spark.table(staging_table_path)
                
                df_filtered = df_staging.join(
                    broadcast(df_processed_ids),
                    "execution_id",
                    "left_anti"
                )

                dfs_to_union.append(df_filtered)
                
            else:
                print(f"  > AVISO: No se encontró la tabla {staging_table_path}")

        # 3. Unificar DataFrames
        if not dfs_to_union:
            print("No hay datos nuevos en ninguna tabla de staging.")
            return 0
        
        df_staging_completo: DataFrame = dfs_to_union[0]
        for df in dfs_to_union[1:]:
            df_staging_completo = df_staging_completo.unionByName(df, allowMissingColumns=True)
        df_staging_completo = df_staging_completo.distinct()

        total_rows = df_staging_completo.count()

        # 4. Append a tabla final
        if total_rows > 0:
            df_staging_completo.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable(EVIDENCES_TABLE)
        else:
            print("No se encontraron registros nuevos para añadir a la tabla final.")
            return 0

        # 5. Limpieza segura de staging
        for table_path in staging_table_paths:
            if spark.catalog.tableExists(table_path):
                try:                  
                    spark.sql(f"DROP TABLE {table_path}")
                    print(f"  > Tabla eliminada correctamente: {table_path}")
                except Exception as e:
                    print(f"  > AVISO: Fallo al limpiar {table_path}: {e}")
    
    except Exception as e:
        print(f"Error fatal durante la unificación de evidencias: {e}")
        raise e
    
    finally:
        print("Proceso completado")
    
    return total_rows

# 3. Punto de Entrada de Ejecución
if __name__ == "__main__":
    rows_processed = 0
    try:
        rows_processed = main()
    except Exception as e:
        raise RuntimeError(f"Fallo en la unificación de evidencias: {e}")
    else:
        dbutils.notebook.exit(f"Éxito: {rows_processed} evidencias nuevas unificadas y añadidas.")
